In [1]:
# At the very top of your notebook—before any asyncio code:
import asyncio
from asyncio import WindowsProactorEventLoopPolicy

asyncio.set_event_loop_policy(WindowsProactorEventLoopPolicy())


In [2]:
# Cell 1 — imports, setup & maps

import json
import logging
import asyncio
import nest_asyncio
from playwright.async_api import async_playwright

nest_asyncio.apply()   # allow nested event loops in Jupyter

# Load your maps
with open("npcs_minified.json") as f:
    NPC_MAP = {n["_id"]: n["name"] for n in json.load(f)}
with open("items.json") as f:
    ITEM_MAP = {i["id"]: i["name"] for i in json.load(f)}

# Opcodes you care about
OP_HIT_EVENT  = 8
OP_DROP_EVENT = 5

# Logging
logging.basicConfig(
    filename="combat.log",
    filemode="a",
    format="%(asctime)s %(message)s",
    datefmt="%H:%M:%S",
    level=logging.INFO
)
logger = logging.getLogger()


In [3]:
# Cell 2 — the async main

async def main():
    # Start Playwright
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()
        page = await context.new_page()

        # Intercept WebSocket frames
        def handle_frame(frame):
            payload = frame.payload
            if not payload.startswith("42"):
                return
            event, batch = json.loads(payload[2:])
            for op, pl in batch:
                if op == OP_HIT_EVENT:
                    a, d, dmg = pl
                    line = f"{NPC_MAP.get(a,'Player')} hit {NPC_MAP.get(d,'Player')}: {dmg}"
                elif op == OP_DROP_EVENT:
                    _, item_id, qty, *_ = pl
                    line = f"→ Drop: {ITEM_MAP.get(item_id,f'Item#{item_id}')} x{qty}"
                else:
                    continue
                print(line)
                logger.info(line)

        # Set up listener for any WebSocket
        context.on("websocket", lambda ws: ws.on("framereceived", handle_frame))

        # Go to the game URL (make sure you’re logged in already)
        await page.goto("https://server1.highspell.com:8888")

        # Keep running
        await asyncio.Future()  # run forever

# Kick it off
await main()


NotImplementedError: 